In [1]:
import pandas as pd
import numpy as np

from keras import backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

2023-01-27 11:23:36.370000: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


We need to read in the cleaned data file and convert to snippets at each Z (one sentence).

We do this based on the tutorial in: https://towardsdatascience.com/a-deep-learning-approach-in-predicting-the-next-word-s-7b0ee9341bfe


In [2]:
corpus = []
with open('cleared_data/cleared.txt') as f:
    lines = f.readlines()
for line in lines:
    for l in line.split("Z"):
        corpus.append(str(l + "Z").strip())


In [3]:
print(len(corpus))
print(type(corpus))
print(corpus[:2])

2006467
<class 'list'>
['MX1+8 MX1+6 MX1+1 MX10 MY1+9 MY1+7 MY1+6 MY1+1 QX1+1 QY1-2 QY1-1 QY10 QX2+2 QX20 QY2-3 QY2-2 QX1+3 QX1+1 QX10 QY1-4 QY1-2 QX2+5 QX2+3 QX2+2 QY2-5 QY2-1 QY20 QX1+7 QX1+4 QX1+2 QX1+1 QX10 QY1-6 QY1-4 QY1-2 QY1-1 QY10 QX2+7 QX2+6 QX2+5 QX2+4 QX2+3 QX2+1 QX20 QY2-7 QY2-4 QY20 QX1+8 QX1+4 QX1+2 QX10 QY1-7 QY1-5 QY10 QX2+8 QX2+5 QX2+2 QX2+1 QY2-7 QY2-5 QY2-1 QY20 QX1+8 QX1+5 QX1+4 QY1-7 QY1-5 QY10 QX2+8 QX2+5 QX2+4 QX2+2 QY2-7 QY2-4 QY2-1 QY20 QX1+8 QX1+5 QX1+4 QX1+3 QX1+2 QX1+1 QY1-7 QY1-1 QX2+8 QX2+5 QX2+1 QY2-6 QY2-4 QY2-2 QY20 QX1+7 QX1+6 QX1+5 QX1+4 QX1+3 QX1+2 QY1-4 QY1-2 QY1-1 QX2+3 QX2+1 QX20 QY2+3 QY2+1 QY20 QX1-1 QY1+3 QY1+2 QY10 Z', 'MX1+8 MX1+5 MX1+4 MX1+3 MY1+9 MY1+4 MY1+3 MY1+2 MY10 QX1+1 QY1-2 QY1-1 QX2+2 QY2-3 QY2-1 QX1+3 QY1-4 QY10 QX2+5 QX2+1 QX20 QY2-4 QY2-3 QY2-2 QY20 QX1+7 QX1+3 QX1+2 QX1+1 QX10 QY1-6 QY1-4 QX2+7 QX2+6 QX2+5 QX2+4 QX2+3 QX2+1 QX20 QY2-7 QY2-4 QX1+8 QX1+4 QX1+2 QY1-7 QY1-5 QY10 QX2+8 QX2+5 QX2+2 QX2+1 QY2-7 QY2-5 QY20 QX1+8 QX1+5 

In [4]:
tokenizer = Tokenizer(filters='\t\n', lower=False)
tokenizer.fit_on_texts(corpus)

In [ ]:
word_index = tokenizer.word_index
total_unique_words = len(tokenizer.word_index) + 1 
print(total_unique_words)
##print(word_index)

In [ ]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)): 
        n_gram_seqs = token_list[:i+1]
        input_sequences.append(n_gram_seqs)
print(len(input_sequences))
#print(input_sequences)

In [ ]:
max_seq_length = max([len(x) for x in input_sequences])
input_seqs = np.array(pad_sequences(input_sequences, maxlen=max_seq_length, padding='pre'))
print(max_seq_length)
print(input_seqs[:5])

In [ ]:
x_values, labels = input_seqs[:, :-1], input_seqs[:, -1]
y_values = tf.keras.utils.to_categorical(labels, num_classes=total_unique_words)
print(x_values[:3])
print(labels[:3])